In [3]:
import os
import warnings
import findfile
import pandas as pd

In [4]:
pip install transformers==4.28.0

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\hongk\OneDrive\Documents\Bangkit\final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv\Scripts\python.exe -m pip install --upgrade pip' command.
c:\users\hongk\OneDrive\Documents\Bangkit\final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
c:\users\hongk\OneDrive\Documents\Bangkit\final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
c:\users\hongk\OneDrive\Documents\Bangkit\final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return 

In [5]:
import transformers
print(transformers.__version__)

4.28.0


In [6]:
import torch

In [7]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

AssertionError: Torch not compiled with CUDA enabled

In [8]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\hongk\OneDrive\Documents\Bangkit\final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv\Scripts\python.exe -m pip install --upgrade pip' command.
c:\users\hongk\OneDrive\Documents\Bangkit\final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
c:\users\hongk\OneDrive\Documents\Bangkit\final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
c:\users\hongk\OneDrive\Documents\Bangkit\final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return 

In [9]:
from pyabsa import ABSAInstruction as absa_instruction

No CUDA GPU found in your device
Version 0.9.9 of metric_visualizer is outdated. Version 0.9.10 was released Sunday January 21, 2024.
[2024-03-15 13:09:54] (2.3.3) PyABSA(2.3.3): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






In [10]:
warnings.filterwarnings("ignore")

In [11]:
task_name = "multitask"
experiment_name = "instruction"
model_checkpoint = "google/flan-t5-base"

In [12]:
print("Experiment Name: ", experiment_name)
model_out_path = "checkpoints"
model_out_path = os.path.join(
    model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}"
)
print("Model output path: ", model_out_path)

Experiment Name:  instruction
Model output path:  checkpoints\multitask\googleflan-t5-base-instruction


In [14]:
id_train_file_path = "./integrated_datasets/acos_datasets/Restaurant-quadruple"
id_test_file_path = "./integrated_datasets/acos_datasets/Restaurant-quadruple"

In [15]:
id_tr_df = absa_instruction.data_utils.read_json(id_train_file_path, "train")
id_te_df = absa_instruction.data_utils.read_json(id_test_file_path, "test")

./integrated_datasets/acos_datasets/Restaurant-quadruple\rest16_quad_train.tsv.jsonl
./integrated_datasets/acos_datasets/Restaurant-quadruple\rest16_quad_test.tsv.jsonl


In [16]:
id_tr_df = pd.DataFrame(id_tr_df)
id_te_df = pd.DataFrame(id_te_df)

In [17]:
loader = absa_instruction.data_utils.InstructDatasetLoader(id_tr_df, id_te_df)

In [18]:
if loader.train_df_id is not None:
    loader.train_df_id = loader.prepare_instruction_dataloader(loader.train_df_id)
if loader.test_df_id is not None:
    loader.test_df_id = loader.prepare_instruction_dataloader(loader.test_df_id)
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.prepare_instruction_dataloader(loader.train_df_ood)
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.prepare_instruction_dataloader(loader.test_df_ood)

In [19]:
# Create T5 utils object
t5_exp = absa_instruction.model.T5Generator(model_checkpoint)

[2024-03-15 13:14:50] (2.3.3) Checkpoint:google/flan-t5-base is not found, you can raise an issue for requesting shares of checkpoints
[2024-03-15 13:14:50] (2.3.3) No checkpoint found in Model Hub for task: google/flan-t5-base


In [20]:
# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.create_datasets(
    t5_exp.tokenize_function_inputs
)

Map:   0%|          | 0/5601 [00:00<?, ? examples/s]

Map:   0%|          | 0/2135 [00:00<?, ? examples/s]

In [21]:
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.create_datasets(
    t5_exp.tokenize_function_inputs
)

Map:   0%|          | 0/5601 [00:00<?, ? examples/s]

Map:   0%|          | 0/2135 [00:00<?, ? examples/s]

In [22]:
# Training arguments
training_args = {
    "output_dir": model_out_path,
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "learning_rate": 5e-5,
    "per_device_train_batch_size": 6,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 6,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "load_best_model_at_end": True,
    "push_to_hub": False,
    "eval_accumulation_steps": 1,
    "predict_with_generate": True,
    "logging_steps": 1000000000,
    "use_mps_device": False,
    # 'fp16': True,
    "fp16": False,
}


In [23]:
# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

Trainer device: cpu

Model training started ....


  0%|          | 0/5604 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

In [ ]:
# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(
    predictor=model_trainer,
    tokenized_dataset=id_tokenized_ds,
    sample_set="train",
    batch_size=16,
)
id_tr_labels = [i.strip() for i in id_ds["train"]["labels"]]

NameError: name 't5_exp' is not defined

In [18]:
id_te_pred_labels = t5_exp.get_labels(
    predictor=model_trainer,
    tokenized_dataset=id_tokenized_ds,
    sample_set="test",
    batch_size=16,
)
id_te_labels = [i.strip() for i in id_ds["test"]["labels"]]

Prediction from trainer


  0%|          | 0/134 [00:00<?, ?it/s]

In [10]:
metrics = t5_exp.get_classic_metrics(id_tr_labels, id_tr_pred_labels)
print("----------------------- Classic Training Set Metrics -----------------------")
print(metrics)

NameError: name 't5_exp' is not defined

In [22]:
metrics = t5_exp.get_classic_metrics(id_te_labels, id_te_pred_labels)
print("----------------------- Classic Testing Set Metrics -----------------------")
print(metrics)

NULL
NULL
NULL:positive
NULL:positive
NULL:yum
NULL:yum
sushi
sushi
sushi:positive
sushi:positive
sushi:good
sushi:good
sushi:FOOD#QUALITY
sushi:FOOD#QUALITY
portions
portions
portions:neutral
portions:neutral
portions:notthebiggest|portions:adequate
portions:notthebiggest|portions:adequate
portions:FOOD#STYLE_OPTIONS|portions:FOOD#STYLE_OPTIONS
portions:FOOD#STYLE_OPTIONS|portions:FOOD#STYLE_OPTIONS
greenteacremebrulee
greenteacremebrulee
greenteacremebrulee:positive
greenteacremebrulee:positive
greenteacremebrulee:must
greenteacremebrulee:must
greenteacremebrulee:FOOD#QUALITY
greenteacremebrulee:FOOD#QUALITY
sushi|service
sushi|service
sushi:positive|service:positive
sushi:positive|service:positive
sushi:great|service:better
sushi:great|service:better
sushi:FOOD#QUALITY|service:SERVICE#GENERAL
sushi:FOOD#QUALITY|service:SERVICE#GENERAL
staff
staff
staff:positive
staff:positive
staff:accomodating
staff:accomodating
staff:SERVICE#GENERAL
staff:SERVICE#GENERAL
restaurant
restaurant
rest